In [ ]:
" Import the libraries " 

import os
import sys 
import math
import copy

import numpy as np
import pandas as pd

from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [ ]:
" Import the scripts of SD for Explaining "

absFilePath = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
newPath = os.path.join(absFilePath, 'SplitSD4X\\')
sys.path.append(newPath)

newPath_supp = os.path.join(newPath, 'supplementary')
sys.path.append(newPath_supp)

from performances import *

In [ ]:
" Import matplotlib and sns to display graphs and figures "

import matplotlib
from matplotlib import pyplot as plt
from matplotlib import transforms
from matplotlib.lines import Line2D
import seaborn as sns
%matplotlib inline

In [ ]:
" Import the plot functions"
sys.path.append(os.path.dirname(os.getcwd()))
from plotFunctions.PlotMSENeighborGen import *
from plotFunctions.PlotMSENbNeighbors import *
from plotFunctions.PlotMSEDiscr import *

In [ ]:
" Define the functions to save and load data "
import pickle
def save_obj(obj, name):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

## 1. Test the effectiveness of the neighbors generation approaches  

In [ ]:
" LOAD THE DATA "

path = './saved_data/'

data_train   = load_obj(path + 'data_train')
target_train = load_obj(path + 'target_train')
data_test    = load_obj(path  + 'data_test')
target_test  = load_obj(path + 'target_test')

all_lists_neigh = []
all_lists_neigh.append(load_obj(path   + 'list_neighbors_1'))
all_lists_neigh.append(load_obj(path   + 'list_neighbors_2'))
all_lists_neigh.append(load_obj(path   + 'list_neighbors_3'))
all_lists_neigh.append(load_obj(path   + 'list_neighbors_4'))

all_lists_subgroups = []
all_lists_subgroups.append(load_obj(path + 'list_subgroups_1'))
all_lists_subgroups.append(load_obj(path + 'list_subgroups_2'))
all_lists_subgroups.append(load_obj(path + 'list_subgroups_3'))
all_lists_subgroups.append(load_obj(path + 'list_subgroups_4'))

" Number of instances to explain "
n = np.size(data_test,0) 

" Number of neighbors "
nb_neighs = np.size(all_lists_neigh[0][0],0)

### Train the Model

In [ ]:
" using Sklearn MLP regressor as black box: "

mlp = make_pipeline(StandardScaler(),
                    MLPRegressor(hidden_layer_sizes=(50, 50),
                                 tol=1e-2, 
                                 max_iter=1000, 
                                 random_state=0))
model_nt = mlp.fit(data_train, target_train)
target_pred_nt = model_nt.predict(data_test)

####  MSE Error

In [ ]:
path = './data_to_plot/'
f_mse = open(path + "mse_generation.txt","w")
f_mse.close()

In [ ]:
nb_models_max = 100
f_mse = open(path + "mse_generation.txt","a")

for j in range (0,4) :
    f_mse.write("Generation_"+str(j+1)+"\n")
    list_subgroups = all_lists_subgroups[j]
    list_neigh = all_lists_neigh[j]
    for i in range (0,nb_models_max-1) :
        S = list_subgroups[i]
        sse_S = loss_sd (S,data_test,list_neigh,model_nt)
        mse_S = sse_S / (n * nb_neighs)
        f_mse.write('{:.2e}'.format(mse_S)+'\n')
f_mse.close()

####  *MSE Error Graph*

In [ ]:
plotMSEwrtNbModels('parkinsons',path)

## 2. Test the influence of the number of neighbors 

In [ ]:
" LOAD THE DATA "

path = './saved_data/'

data_train   = load_obj(path + 'data_train_n')
target_train = load_obj(path + 'target_train_n')
data_test    = load_obj(path  + 'data_test_n')
target_test  = load_obj(path + 'target_test_n')

all_lists_neigh = []
all_lists_neigh.append(load_obj(path   + 'list_neighbors_20'))
all_lists_neigh.append(load_obj(path   + 'list_neighbors_50'))
all_lists_neigh.append(load_obj(path   + 'list_neighbors_100'))

all_lists_subgroups = []
all_lists_subgroups.append(load_obj(path + 'list_subgroups_20'))
all_lists_subgroups.append(load_obj(path + 'list_subgroups_50'))
all_lists_subgroups.append(load_obj(path + 'list_subgroups_100'))

" Number of instances to explain "
n = np.size(data_test,0) 

" Number of neighbors "
nb_neighs = [20,50,100]

### Train the Model

In [ ]:
" using Sklearn MLP regressor as black box: "

mlp = make_pipeline(StandardScaler(),
                    MLPRegressor(hidden_layer_sizes=(50, 50),
                                 tol=1e-2, 
                                 max_iter=1000, 
                                 random_state=0))
model_nt = mlp.fit(data_train, target_train)
target_pred_nt = model_nt.predict(data_test)

####  MSE Error

In [ ]:
path = './data_to_plot/'
f_mse = open(path + "mse_nb_neighs.txt","w")
f_mse.close()

In [ ]:
nb_models_max = 100
f_mse = open(path + "mse_nb_neighs.txt","a")

for j in range (0,3) :
    f_mse.write("Generation_"+str(j+1)+"\n")
    list_subgroups = all_lists_subgroups[j]
    list_neigh = all_lists_neigh[j]
    for i in range (0,nb_models_max-1) :
        S = list_subgroups[i]
        sse_S = loss_sd (S,data_test,list_neigh,model_nt)
        mse_S = sse_S / (n * nb_neighs[j])
        f_mse.write('{:.2e}'.format(mse_S)+'\n')
f_mse.close()

In [ ]:
plotMSENbNeighbors('parkinsons',path)

## 3. Test the parameter K of discretization  

In [ ]:
" LOAD THE DATA "

path = './saved_data/'

data_train   = load_obj(path + 'data_train_d')
target_train = load_obj(path + 'target_train_d')
data_test    = load_obj(path  + 'data_test_d')
target_test  = load_obj(path + 'target_test_d')
list_neigh = load_obj(path   + 'list_neighbors_d')

L_Subgroups_freq = load_obj(path + 'l_list_subgroups_freq')
L_Subgroups_width = load_obj(path + 'l_list_subgroups_width')

" Number of instances to explain "
n = np.size(data_test,0) 

" Number of neighbors "
nb_neighs = np.size(list_neigh[0],0)

### Train the Model

In [ ]:
" using Sklearn MLP regressor as black box: "

mlp = make_pipeline(StandardScaler(),
                    MLPRegressor(hidden_layer_sizes=(50, 50),
                                 tol=1e-2, 
                                 max_iter=1000, 
                                 random_state=0))
model_nt = mlp.fit(data_train, target_train)
target_pred_nt = model_nt.predict(data_test)

### 3.1  Equal Frequency  

####  MSE Error

In [ ]:
path = './data_to_plot/'
f_mse = open(path + "mse_disc_freq.txt","w")
f_mse.close()

In [ ]:
nb_models_max = 100
f_mse = open(path + "mse_disc_freq.txt","a")

for j in range (0,len(L_Subgroups_freq)) :
    f_mse.write("Generation_"+str(j+1)+"\n")
    list_subgroups = L_Subgroups_freq[j]
    for i in range (0,nb_models_max-1) :
        S = list_subgroups[i]
        sse_S = loss_sd (S,data_test,list_neigh,model_nt)
        mse_S = sse_S / (n * nb_neighs)
        f_mse.write('{:.2e}'.format(mse_S)+'\n')
f_mse.close()

In [ ]:
plotMSEDiscFreq('parkinsons',path)

### 3.1  Equal Width  

In [ ]:
path = './data_to_plot/'
f_mse = open(path + "mse_disc_width.txt","w")
f_mse.close()

In [ ]:
nb_models_max = 100
f_mse = open(path + "mse_disc_width.txt","a")

for j in range (0,len(L_Subgroups_width)) :
    f_mse.write("Generation_"+str(j+1)+"\n")
    list_subgroups = L_Subgroups_width[j]
    for i in range (0,nb_models_max-1) :
        S = list_subgroups[i]
        sse_S = loss_sd (S,data_test,list_neigh,model_nt)
        mse_S = sse_S / (n * nb_neighs)
        f_mse.write('{:.2e}'.format(mse_S)+'\n')
f_mse.close()

In [ ]:
plotMSEDiscWidth('parkinsons',path)